In [2]:
import casadi

In [63]:
class d_test_fn(casadi.Callback):
  def __init__(self, name, opts={}):
    casadi.Callback.__init__(self)
    self.construct(name, opts)

  # Number of inputs and outputs
  def get_n_in(self): return 2
  def get_n_out(self): return 1

  def get_sparsity_in(self, n_in): return casadi.Sparsity.dense(3)
  def get_sparsity_out(self, n_out): return casadi.Sparsity.dense(3,3) 

  # def has_jacobian(self, *args) -> bool:
  #   return True
  # def get_jacobian(self, *args):
  #   print(args)
  #   return self.jac(*args)

  # Initialize the object
  def init(self):
    # print('initializing object')
    pass

  # Evaluate numerically
  def eval(self, arg):
    return [casadi.DM.eye(3)]
    
class test_fn(casadi.Callback):
  def __init__(self, name, opts={}):
    casadi.Callback.__init__(self)
    # idk how relevant this is but whatever
    self.construct(name, opts)
    self.df = d_test_fn('df')

  # define number of inputs  
  def get_n_in(self): return 1
  # define number of outputs
  def get_n_out(self): return 1

  # define shape of input number n_in
  def get_sparsity_in(self, n_in): return casadi.Sparsity.dense(3)
  # define shape of output number n_out
  def get_sparsity_out(self, n_out): return casadi.Sparsity.dense(3)

  # tell opti that fcn has a derivative
  def has_jacobian(self): return True
  def get_jacobian(self, name, inames, onames, opts={}):
    x = casadi.MX.sym(inames[0], 3)
    out = casadi.MX.sym(inames[1],3)
    jac = self.df(x, out)
    # need to register the function so it doesn't go out of scope when returning
    return casadi.Function(name, [x,out], [jac])

  def eval(self, arg):
    # arguments are given as a list
    x = arg[0]
    # print(x, type(x))
    # casadi expects return to be in a vector
    return [x]


In [64]:
f = test_fn('f')
df = d_test_fn('f_jac')

print(f([0,0,0]))

[0, 0, 0]


In [58]:
opti = casadi.Opti()
x = opti.variable(3)
opts = {}
# opts['ipopt.hessian_approximation'] = 'limited-memory'
opti.minimize(x[0]**2 + x[1]**2 + x[2]**2)
opti.subject_to(x[0]+x[1]+x[2] == 1)

opti.solver("ipopt", opts)
sol=opti.solve()
print(sol)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.00e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [65]:
opti = casadi.Opti()
x = opti.variable(3)
opts = {}
# neccesary to avoid hessian computations that we cannot produce with mujoco setup
opts['ipopt.hessian_approximation'] = 'limited-memory'
opti.minimize(f(x)[0]**2 + f(x)[1]**2+f(x)[2]**2)
opti.subject_to(f(x)[0] + f(x)[1] == f(x)[2] +1)

opti.solver("ipopt", opts)
sol=opti.solve()
print(sol)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.00e+00 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [66]:
sol.value(x)

array([ 0.33333333,  0.33333333, -0.33333333])